### Dataset Preperation
We need the prepare the dataset with both stats and form. we will apply a rolling window on every player and then add those matches data as 5 more columns in the data. Then we can move onto Some features and Engineering. 

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('Filled_data.csv')
df.head()

,Unnamed: 0,Match Runs,Match SR,Total Runs,Total Balls,Ave,Career SR,HS,NO,0,50,100,Opposition,Ground,match_id,player_id
0,0,0,0.00,0,2,0.0,0.00,0,0,1,0,0,Pakistan,Gujranwala,593,35320
1,1,0,0.00,0,4,0.0,0.00,0,0,2,0,0,New Zealand,Dunedin,612,35320
2,2,36,92.30,36,43,12.0,83.72,36,0,2,0,0,New Zealand,Wellington,616,35320
3,3,10,83.33,46,55,11.5,83.63,36,0,2,0,0,Sri Lanka,Sharjah,623,35320
4,4,20,80.00,66,80,13.2,82.50,36,0,2,0,0,Pakistan,Sharjah,625,35320


We are seperating the data of each player so we dont cross over in our rolling window

In [16]:
runs = []
for i in df['player_id'].unique():
    player = df[df['player_id'] == i]['Match Runs'].tolist()
    runs.append(player)

Using numpy's sliding window function we get the runs in the format we need

In [17]:
runs_cleaned = []
for run in runs:
    runs_cleaned.append(np.lib.stride_tricks.sliding_window_view(run, 5))

Then we need to remove the initial 5 elements from every batsmans list because we dont have enough datapoints for them to be included in the dataset. We get the indexes for every player's seperation in the dataframe and then drop the rows from dataframe

In [18]:
idx = []
players = df['player_id'].unique().tolist()
for i in players:
    idx.append((df['player_id']==i).idxmax())

In [19]:
indexes = []
for i in idx:
    indexes.append(i)
    indexes.append(i+1)
    indexes.append(i+2)
    indexes.append(i+3)
    indexes.append(i+4)

In [20]:
df.drop(indexes, inplace=True)
df.head()

,Unnamed: 0,Match Runs,Match SR,Total Runs,Total Balls,Ave,Career SR,HS,NO,0,50,100,Opposition,Ground,match_id,player_id
5,5,19,54.28,85,115,14.16,73.91,36,0,2,0,0,England,Leeds,634,35320
6,6,31,119.23,116,141,16.57,82.26,36,0,2,0,0,England,Nottingham,635,35320
7,7,36,163.63,152,163,19.00,93.25,36,0,2,0,0,Sri Lanka,Nagpur,644,35320
8,8,53,129.26,205,204,22.77,100.49,53,0,2,1,0,Sri Lanka,Pune,646,35320
9,9,30,103.44,235,233,23.50,100.85,53,0,2,1,0,Sri Lanka,Margao,648,35320


We now remove the last row from each sliding window as the last 5 matches don't have next value to predict

In [21]:
runs = []
for run in runs_cleaned:
    l = len(run)
    j=0
    for i in run:
        if(j==l-1):
            continue
        runs.append(i)
        j+=1
runs = np.asarray(runs)

Adding columns to the dataframe for each of the 5 matches

In [22]:
df['m1'] = runs[:, 0]
df['m2'] = runs[:, 1]
df['m3'] = runs[:, 2]
df['m4'] = runs[:, 3]
df['m5'] = runs[:, 4]

Drop unwanted columns and save the dataset to csv file. 

In [23]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,Match Runs,Match SR,Total Runs,Total Balls,Ave,Career SR,HS,NO,0,50,100,Opposition,Ground,match_id,player_id,m1,m2,m3,m4,m5
5,19,54.28,85,115,14.16,73.91,36,0,2,0,0,England,Leeds,634,35320,0,0,36,10,20
6,31,119.23,116,141,16.57,82.26,36,0,2,0,0,England,Nottingham,635,35320,0,36,10,20,19
7,36,163.63,152,163,19.00,93.25,36,0,2,0,0,Sri Lanka,Nagpur,644,35320,36,10,20,19,31
8,53,129.26,205,204,22.77,100.49,53,0,2,1,0,Sri Lanka,Pune,646,35320,10,20,19,31,36
9,30,103.44,235,233,23.50,100.85,53,0,2,1,0,Sri Lanka,Margao,648,35320,20,19,31,36,53


We still need to do some cleaning. The * in the Match runs column needs to be removed. We need to remove match runs from total runs and also from 100, 50, 0 and NO.

In [24]:
df.loc[df['Match Runs'].str.contains('\*'), 'NO'] = df['NO'] - 1
df.dropna(subset=['Match Runs', 'm1', 'm2', 'm3', 'm4', 'm5'], inplace=True)
df["Match Runs"] = df["Match Runs"].str.extract('(\d+)', expand=False).astype(float)
df["m1"] = df["m1"].str.extract('(\d+)', expand=False).astype(float)
df["m2"] = df["m2"].str.extract('(\d+)', expand=False).astype(float)
df["m3"] = df["m3"].str.extract('(\d+)', expand=False).astype(float)
df["m4"] = df["m4"].str.extract('(\d+)', expand=False).astype(float)
df["m5"] = df["m5"].str.extract('(\d+)', expand=False).astype(float)
df["HS"] = df["HS"].str.extract('(\d+)', expand=False).astype(float)

In [25]:
df['Total Runs'] = df['Total Runs'] - df['Match Runs']
# subtract 1 from column 100 if Match Runs>=100
df.loc[df['Match Runs'] >= 100, '100'] = df['100'] - 1
df.loc[(df['Match Runs'] >= 50) & (df['Match Runs'] < 100), '50'] = df['50'] - 1
df.loc[df['Match Runs'] == 0, '0'] = df['0'] - 1

In [26]:
df.to_csv('dataset.csv', index=False)